In [1]:
from azureml.core import Environment

env = Environment(name="azure_pytorch_env")
env.python.conda_dependencies.add_pip_package("torch==2.1.0")
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("scikit-learn")


In [2]:
from azureml.core import Workspace, Model, Environment
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Connect to Azure ML Workspace
ws = Workspace.from_config()

# Load the registered model
model = Model(ws, "breast_cancer_nn")

# Define inference configuration
inference_config = InferenceConfig(
    entry_script="score.py",
    environment=env
)

# Define deployment configuration (ACI)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model
service = Model.deploy(ws, "breast-pytorch", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

# Print Scoring URI
print(f"Deployment successful! Scoring URI: {service.scoring_uri}")


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
/tmp/ipykernel_55840/3104958579.py:21: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, "breast-pytorch", [model], inference_config, deployment_config)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-03-29 16:48:13+05:45 Creating Container Registry if not exists.
2025-03-29 16:48:15+05:45 Use the existing image.
2025-03-29 16:48:17+05:45 Submitting deployment to compute.
2025-03-29 16:48:23+05:45 Checking the status of deployment breast-pytorch..
2025-03-29 16:51:08+05:45 Checking the status of inference endpoint breast-pytorch.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Deployment successful! Scoring URI: http://6ee4446e-b3c5-443f-8f3b-deedb34f0219.eastus2.azurecontainer.io/score


In [4]:
import requests
import json
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

# Load dataset and preprocess one sample for testing

X, y = load_breast_cancer(return_X_y=True)

# Select a sample (first one for testing)
sample_input = X[-55].tolist()

# Define the input JSON payload
payload = json.dumps({"data": [sample_input]})

# Get the deployment endpoint
scoring_uri = service.scoring_uri
headers = {"Content-Type": "application/json"}

# Send request to the deployed model
response = requests.post(scoring_uri, data=payload, headers=headers)

# Print response
print("Response:", response.json())


Response: {"predictions": [[0]]}
